In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import math
from sklearn import linear_model
from scipy import stats

In [ ]:
DATA_CSV = 'data/ME/ME_2a.csv'

In [ ]:
data_points = pd.read_csv(DATA_CSV)

In [ ]:
nr1 = data_points['carb']
nr1 = nr1.iloc[1:len(nr1)]
nr2 = data_points['py']
nr2 = nr2.iloc[1:len(nr2)]
nr3 = data_points['human_p']
nr3 = nr3.iloc[1:len(nr3)]
nr4 = data_points['human_n']
nr4 = nr4.iloc[1:len(nr4)]

In [ ]:
data_points = data_points.iloc[0:len(data_points.index)-1]
nr1.index = np.arange(len(nr1.index))
nr2.index = np.arange(len(nr2.index))
nr3.index = np.arange(len(nr3.index))
nr4.index = np.arange(len(nr4.index))

In [ ]:
data_points.loc[:,'post_py'] = nr2
data_points.loc[:,'human_pp'] = nr3
data_points.loc[:,'human_nn'] = nr4
data_points.loc[:,'growth'] = (nr1 - data_points['carb']) / (data_points['post_py'] - data_points['py'])
data_points = data_points[data_points.post_py - data_points.py == 5]
data_points.drop(['py','post_py'], axis=1, inplace=True)
data_points.drop(['human_p','human_n'], axis=1, inplace=True)
data_points.index = np.arange(len(data_points.index))

In [ ]:
data_points = data_points.loc[:,['growth','carb','lon','lat']]

In [ ]:
points = []
for i, point in data_points.iterrows():
    points.append([np.array(point.drop('growth')).tolist(), point['growth']])

In [ ]:
N_GROUPS = 10
random.shuffle(points)
groups = []
prev_cutoff = 0
for i in np.arange(N_GROUPS):
    next_cutoff = (i + 1) * len(points) / N_GROUPS
    groups.append(points[prev_cutoff:next_cutoff])
    prev_cutoff = next_cutoff

In [ ]:
min_mse = float("inf")
for i in np.arange(N_GROUPS):
    training = []
    test = []
    for j, group in enumerate(groups):
        if j==i:
            test = group
        else:
            training.extend(group)
    training = np.array(training).T.tolist()
    clf = linear_model.LinearRegression()
    clf.fit(training[0], training[1])
    coef = clf.coef_
    cons = clf.intercept_
    sq_error = 0.0
    for elem in test:
        predicted = sum(coef * elem[0]) + cons
        actual = elem[1]
        sq_error += (actual - predicted) ** 2
    mse = math.sqrt(sq_error/len(test))
    if mse < min_mse:
        min_mse = mse

In [ ]:
min_mse